***
# **FloodProofs Labs - HMC Training - TimeSeries Analyzer** 

<img style="float: left; padding-right: 80px; padding-left: 5px;" src="img/logo_hmc.png" width="200px" align=”left” >

In the laboratory of **HMC time-series** we will perform the following points:
   * Configure the libraries and the dependecies of the laboratory;
   * Read the configuration file of the laboratory;
   * Read the static datasets of terrain, river networks and outlet sections;
   * Select the 'time run' and the 'outlet section' to perform the time-series analysis;
   * Read the dynamic datasets of the time-series (collections and hydrographs);
   * Plot the position of the analyzed outlet section;
   * Plot the time-series of discharge and the time-series of the hmc average forcings.    


## **Import libraries and dependencies**

In [1]:
%matplotlib inline
%matplotlib widget

# Libraries
from library.jupyter_generic.lib_jupyter_data_io_json import read_file_settings, read_file_ts_hydrograph
from library.jupyter_generic.lib_jupyter_data_io_generic import define_file_path, define_file_template, \
    fill_file_template, create_dframe_ts, get_path_root, get_path_folders, get_folders_time

from library.jupyter_generic.lib_jupyter_data_geo_ascii import read_data_grid
from library.jupyter_generic.lib_jupyter_data_geo_shapefile import read_data_section, find_data_section

from library.jupyter_generic.lib_jupyter_data_io_netcdf import read_file_ts_collections

from library.jupyter_generic.lib_jupyter_plot_ts import plot_ts_discharge, plot_ts_forcing
from library.jupyter_generic.lib_jupyter_plot_map import plot_map_terrain

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Define configuration file
file_name_settings="fp_labs_analyzer_hmc_timeseries.json"

# Info
print(' ==> Libraries loaded')

 ==> Libraries loaded


### **Configure the flood-proofs laboratory**
- Load the configuration file:

In [2]:
# Read data from settings algorithm file
settings_info = read_file_settings(file_name_settings)

# Info
print(' ==> Settings information loaded')

 ==> Settings information loaded


- Define the static and dynamic file paths:

In [3]:
# Define static and dynamic file path(s)
file_path_dset_static = define_file_path(settings_info['source']['static'])
file_path_dset_dynamic = define_file_path(settings_info['source']['dynamic'])
file_path_dset_plot = define_file_path(settings_info['destination']['plot'])

In [4]:
# Define dynamic path root
file_path_dynamic_root = get_path_root(list(file_path_dset_dynamic.values())[0])

In [5]:
# Define list of available folders
list_folder_dynamic_root = get_path_folders(file_path_dynamic_root)

### **Read the static datasets**
- Read the terrain and river networks files in ascii format

In [6]:
# Read terrain datasets
darray_terrain = read_data_grid(file_path_dset_static['terrain'], var_limit_min=0, var_limit_max=None)

In [7]:
# Read river network datasets
darray_river_network = read_data_grid(file_path_dset_static['river_network'], var_limit_min=0, var_limit_max=1)

- Read the sections information in vector format

In [8]:
# Read sections shapefile
dset_section = read_data_section(file_path_dset_static['sections'])

# Define outlet sections list
outlet_section_list = ['_'.join([sec_domain, sec_name]) for sec_name, sec_domain in zip(dset_section['section_name'], dset_section['section_domain'])] 

### **Select the "time run" and the "outlet section"**
- Select the time run to perform the time-series analysis

In [9]:
# Define list of available runs based on folders name
time_dynamic_list = get_folders_time(list_folder_dynamic_root)

In [10]:
# Display available time run
time_run_selection = widgets.Select(options=time_dynamic_list, description='Time Run', disabled=False)
display(time_run_selection)  

Select(description='Time Run', options=('2021-01-22 06:00',), value='2021-01-22 06:00')

In [11]:
# Parser the selection from the scroll menu
time_run_value = time_run_selection.value

# Set time run in the setting info object
settings_info['time_run'] = time_run_value

# Info time run
print(' ==> Time Run: ' + settings_info['time_run'] )

 ==> Time Run: 2021-01-22 06:00


- Select the outlet section to perform the time-series analysis

In [12]:
# Display available outlet sections
outlet_sections_selection = widgets.Select(options=outlet_section_list, description='Outlet Sections', disabled=False)
display(outlet_sections_selection)   

Select(description='Outlet Sections', options=('Foglia_Foglia3', 'Foglia_Montecchio', 'Foglia_Bronzo', 'Arzill…

In [13]:
# Parser the selection from the scroll menu
outlet_section_value = outlet_sections_selection.value
basin_name, section_name = outlet_section_value.split('_')

# Set section name and basin in the setting info object
settings_info['info']['section_name'] = section_name
settings_info['info']['basin_name'] = basin_name

# Info section
print(' ==> SectionName: ' + settings_info['info']['section_name'] + ' -- BasinName: ' + settings_info['info']['basin_name'] )

 ==> SectionName: Aspio2 -- BasinName: Aspio


- Define the information of section, domain and time

In [14]:
# Get domain, section and time information
info_section = find_data_section(dset_section, section_name=settings_info['info']['section_name'],
                                 basin_name=settings_info['info']['basin_name'])
info_domain = settings_info['info']['domain_name']
info_time_run = settings_info['time_run']

# Fill dynamic file path(s)
file_template_filled = define_file_template(
    info_time_run, section_name=info_section['section_name'], basin_name=info_section['basin_name'],
    domain_name=info_domain, template_default=settings_info['template'])
file_path_dset_dynamic = fill_file_template(file_path_dset_dynamic,
                                            template_filled=file_template_filled,
                                            template_default=settings_info['template'])
file_path_dset_plot = fill_file_template(file_path_dset_plot,
                                         template_filled=file_template_filled,
                                         template_default=settings_info['template'])
# Info
print(' ==> SectionName: ' + info_section['section_name'] + ' -- BasinName: ' + info_section['basin_name'] + ' -- TimeRun: ' + info_time_run)

 ==> SectionName: Aspio2 -- BasinName: Aspio -- TimeRun: 2021-01-22 06:00


### **Read the dynamic datasets**
- Read the collections file in netcdf format

In [15]:
# Read the collections file 
dframe_ts_cls, attrs_ts_cls = read_file_ts_collections(file_path_dset_dynamic['time_series_collections'])
# Info
print(' ==> Collections dynamic file ' + file_path_dset_dynamic['time_series_collections'] + '... LOADED')

 ==> Collections dynamic file /home/fabio/fp_labs_datasets/data_dynamic/20210122_06/data_archive/collections/hmc.collections.202101220600.nc... LOADED


- Read the hydrograph file in json format

In [16]:
# Read the hydrograph file
dframe_ts_cls, attrs_cls = read_file_ts_collections(file_path_dset_dynamic['time_series_collections'])
dframe_ts_hydro, attrs_ts = read_file_ts_hydrograph(file_path_dset_dynamic['time_series_hydrograph'])
# Info
print(' ==> Hydrograph dynamic file ' + file_path_dset_dynamic['time_series_hydrograph'] + ' ... LOADED')

 ==> Hydrograph dynamic file /home/fabio/fp_labs_datasets/data_dynamic/20210122_06/data_archive/collections/hydrograph_Aspio2_Aspio_202101220600.json ... LOADED


- Select the variable(s) and create the dataframe

In [17]:
# Select output variable(s) to plot time-series
dframe_ts_discharge_obs = create_dframe_ts(dframe_ts_hydro, var_name_in='discharge_observed', var_name_out='discharge_observed')
dframe_ts_discharge_sim = create_dframe_ts(dframe_ts_hydro, var_name_in='discharge_simulated', var_name_out='discharge_simulated')
dframe_ts_rain = create_dframe_ts(dframe_ts_cls, var_name_in='rain', var_name_out='rain')
dframe_ts_sm = create_dframe_ts(dframe_ts_cls, var_name_in='soil_moisture', var_name_out='soil_moisture')

# Select forcing variable(s) to plot time-series
dframe_ts_airt = create_dframe_ts(dframe_ts_cls, var_name_in='air_temperature', var_name_out='air_temperature')
dframe_ts_incrad = create_dframe_ts(dframe_ts_cls, var_name_in='incoming_radiation', var_name_out='incoming_radiation')
dframe_ts_rh = create_dframe_ts(dframe_ts_cls, var_name_in='relative_humidity', var_name_out='relative_humidity')
dframe_ts_wind = create_dframe_ts(dframe_ts_cls, var_name_in='wind', var_name_out='wind')

### **Plot the dynamic datasets in time-series format**
- Create the plot for analyzing the hmc discharges output

In [18]:
# Plot ts discharge
file_name_ts_discharge = file_path_dset_plot['time_series_discharge']
plot_ts_discharge(file_name_ts_discharge, dframe_ts_discharge_sim, attrs_ts,
                  df_discharge_obs=dframe_ts_discharge_obs, df_rain=dframe_ts_rain,
                  df_soil_moisture=dframe_ts_sm)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

- Create the plot for analyzing the hmc forcing datasets

In [19]:
# Plot ts forcing
file_name_ts_forcing = file_path_dset_plot['time_series_forcing']
plot_ts_forcing(file_name_ts_forcing,
                df_rain=dframe_ts_rain, df_airt=dframe_ts_airt, df_incrad=dframe_ts_incrad,
                df_rh=dframe_ts_rh, df_winds=dframe_ts_wind,
                attrs_forcing=attrs_ts)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
# Plot map terrain with section
file_name_section_locator = file_path_dset_plot['section_locator']
plot_map_terrain(file_name_section_locator, darray_terrain, darray_river_network, info_section,
                 mask_terrain=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Training on-the-job**
- Download/Organize the static and dynamic datasets for a different "time run";
- Select a different case-study (time run);
- Select a different outlet section (section name, basin name);
- Add/change the variables in the time-series plot (have to check the names of the variables in the netcdf forcing or outcome files);
- Add/change the plot of terrain datasets (for example using a different map background);
- ...